In [1]:
import warnings
warnings.filterwarnings("ignore")

import scanpy as sc
import squidpy as sq
warnings.filterwarnings("ignore")

# import module
import stlearn as st
from pathlib import Path
st.settings.set_figure_params(dpi=300)


In [2]:
data = sq.datasets.slideseqv2() # 4000高变基因
data.var_names_make_unique() 
data = data[(data.obs["x"]>=1000) & (data.obs["x"]<=5600) & (data.obs["y"]<=5400) & (data.obs["y"]>=2600)]

In [3]:
data

View of AnnData object with n_obs × n_vars = 26146 × 4000
    obs: 'barcode', 'x', 'y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MT', 'log1p_total_counts_MT', 'pct_counts_MT', 'n_counts', 'leiden', 'cluster'
    var: 'MT', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cluster_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'spatial_neighbors', 'umap'
    obsm: 'X_pca', 'X_umap', 'deconvolution_results', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances', 'spatial_connectivities', 'spatial_distances'

In [4]:
st.pp.filter_genes(data,min_cells=1)
st.pp.normalize_total(data)
st.pp.log1p(data)

Normalization step is finished in adata.X
Log transformation step is finished in adata.X


In [5]:
data

AnnData object with n_obs × n_vars = 26146 × 4000
    obs: 'barcode', 'x', 'y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MT', 'log1p_total_counts_MT', 'pct_counts_MT', 'n_counts', 'leiden', 'cluster'
    var: 'MT', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cluster_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'spatial_neighbors', 'umap', 'log1p'
    obsm: 'X_pca', 'X_umap', 'deconvolution_results', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances', 'spatial_connectivities', 'spatial_distances'

In [6]:
# run PCA for gene expression data
st.em.run_pca(data,n_comps=50)

PCA is done! Generated in adata.obsm['X_pca'], adata.uns['pca'] and adata.varm['PCs']


In [7]:
data_SME = data.copy()
# apply stSME to normalise log transformed data
st.spatial.SME.SME_normalize(data_SME, use_data="raw")
data_SME.X = data_SME.obsm['raw_SME_normalized']
st.pp.scale(data_SME)
st.em.run_pca(data_SME,n_comps=50)

KeyError: 'imagerow'

In [ ]:
# pre-processing for spot image
st.pp.tiling(data, TILE_PATH)

# this step uses deep learning model to extract high-level features from tile images
# may need few minutes to be completed
st.pp.extract_feature(data)

In [ ]:
from scanpy import read_10x_h5

# proj_list = ['151507']
ARI_list,AMI_list = [],[]

# adata = sq.datasets.slideseqv2() # 4000高变基因
# adata.var_names_make_unique() 
# adata = adata[(adata.obs["x"]>=1000) & (adata.obs["x"]<=5600) & (adata.obs["y"]<=5400) & (adata.obs["y"]>=2600)]
adata = read_10x_h5("/home/tengliu/Torch_pyG/Breast_cancer_comparison/SPGDL/data/V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5")
spatial=pd.read_csv("/home/tengliu/Torch_pyG/Breast_cancer_comparison/SPGDL/data/spatial/tissue_positions_list.csv",sep=",",header=None,na_filter=False,index_col=0) 

adata.obs["x1"]=spatial[1]
adata.obs["x2"]=spatial[2]
adata.obs["x3"]=spatial[3]
adata.obs["x4"]=spatial[4]
adata.obs["x5"]=spatial[5]
adata.obs["x_array"]=adata.obs["x2"]
adata.obs["y_array"]=adata.obs["x3"]
adata.obs["x_pixel"]=adata.obs["x4"]
adata.obs["y_pixel"]=adata.obs["x5"]
#Select captured samples
adata=adata[adata.obs["x1"]==1]
adata.var_names=[i.upper() for i in list(adata.var_names)]
adata.var["genename"]=adata.var.index.astype("str")
img=cv2.imread("/home/tengliu/Torch_pyG/Breast_cancer_comparison/SPGDL/data/V1_Breast_Cancer_Block_A_Section_1_image.tif")
#Set coordinates
x_array=adata.obs["x_array"].tolist()
y_array=adata.obs["y_array"].tolist()
x_pixel=adata.obs["x_pixel"].tolist()
y_pixel=adata.obs["y_pixel"].tolist()

#Test coordinates on the image
img_new=img.copy()
for i in range(len(x_pixel)):
    x=x_pixel[i]
    y=y_pixel[i]
    img_new[int(x-20):int(x+20), int(y-20):int(y+20),:]=0

#Calculate adjacent matrix
s=1
b=49
adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, x_pixel=x_pixel, y_pixel=y_pixel, image=img, beta=b, alpha=s, histology=True)
#If histlogy image is not available, SpaGCN can calculate the adjacent matrix using the fnction below
#adj=calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)

#Normalization
adata.var_names_make_unique()
spg.prefilter_genes(adata,min_cells=3) # avoiding all genes are zeros
spg.prefilter_specialgenes(adata)
#Normalize and take log for UMI
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

p=0.5 
#Find the l value given p
l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)


n_clusters = 20
#Set seed
r_seed=t_seed=n_seed=100
#Seaech for suitable resolution
res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)

clf=spg.SpaGCN()
clf.set_l(l)
#Set seed
random.seed(r_seed)
torch.manual_seed(t_seed)
np.random.seed(n_seed)
#Run
clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
y_pred, prob=clf.predict()
adata.obs["pred"]= y_pred
adata.obs["pred"]=adata.obs["pred"].astype('category')

# load ground truth read the annotation  
df_meta = pd.read_csv('/home/tengliu/Torch_pyG/Breast_cancer_comparison/SPGDL/data/metadata.tsv', sep='\t',index_col=0)
# index_col = 0 能够去掉dataframe最左边的序号列。
df_meta = df_meta.drop(columns=['annot_type'])
df_meta.columns = ['Ground Truth']
adata.obs['Ground Truth'] = df_meta.loc[adata.obs_names, 'Ground Truth']

from sklearn import metrics
obs_df = adata.obs.dropna() ##过滤掉缺失值的行，即当前分类值为Nan，就把该行过滤掉。
ARI = metrics.adjusted_rand_score(obs_df['pred'], obs_df['Ground Truth']) ## ARI 是从 包sklearn.metrics.cluster中导入的衡量标准
AMI = metrics.adjusted_mutual_info_score(obs_df['pred'], obs_df['Ground Truth'])

print('Adjusted rand index = %.2f' %ARI)
adata.uns["ARI"] = ARI
adata.uns["AMI"] = AMI

ARI_list.append(ARI)
AMI_list.append(AMI)

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.pca(adata,n_comps=30)
sc.pp.neighbors(adata)
# sc.pp.neighbors(adata, use_rep='STAGATE',n_neighbors=250,n_pcs=30,knn=True,method="umap") 
# ## STAGATE为训练出来的embeddings, 然后用它进行下游的分析。
sc.tl.umap(adata) ## 利用neighbors计算降维图Umap.

adata_filename = "Human_Cancer"+"_spaGCN"+".h5ad"
adata.filename = './Results/'+adata_filename

Metrics_spaGCN = pd.DataFrame([ARI_list,AMI_list])
Metrics_spaGCN.to_csv("./Results/Metrics_spaGCN .csv")

In [ ]:
adata

In [ ]:
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.umap(adata, color=['pred', "Ground Truth"],legend_loc='on data', 
            legend_fontsize=12,legend_fontoutline=2,frameon=False, title=['pred' + ' (ARI=%.2f)'%ARI, "Ground Truth"])